## Collecting and analysing data from LinkedIn

In [89]:
from linkedin_api import Linkedin
import pandas as pd  # tabular data
pd.set_option('display.max_columns', None)
from tqdm.auto import tqdm # progress bars
tqdm.pandas()
import json
from pprint import pprint
import time

In [ ]:
secrets = json.load(open("secrets.json"))
api = Linkedin(secrets["username"], secrets["password"])

Sample search: https://www.linkedin.com/search/results/people/?geoUrn=%5B%22105490917%22%5D&origin=FACETED_SEARCH&pastCompany=%5B%221073%22%2C%221038%22%2C%223255299%22%2C%22397575%22%5D&sid=O0Q

KPMG New Zealand: https://www.linkedin.com/company/kpmg-new-zealand/: 1,083 employees, ~2500 past employees  
PwC New Zealand: https://www.linkedin.com/company/pwc-new-zealand/: 1,093 employees, ~1600 past employees  
Deloitte (Worldwide): https://www.linkedin.com/company/deloitte/: 363,547 employees ~1500 in NZ, ~4100 past employees in NZ  
EY (Worldwide): https://www.linkedin.com/company/ernstandyoung/: 319,968 employees, ~1200 in NZ, ~3400 past employees in NZ  

Summative past employees in NZ
~9700

375 people are currently working at one of the 4 and have past worked for one of the 4

In [ ]:
%%time
companies = [
    "3255299", # PwC
    "397575", # KPMG
    "1038", # Deloitte
    "1073" # EY
]
regions = [
    "105490917" # New Zealand
]
people = []
# Get past employees
for company in tqdm(companies):
    people.extend(api.search_people(past_companies=[company], regions=regions))
# Get current employees
for company in tqdm(companies):
    people.extend(api.search_people(current_company=[company], regions=regions))
# Get all past and current PwC and KPMG
for company in tqdm(companies[:2]):
    people.extend(api.search_people(past_companies=[company]))
for company in tqdm(companies[:2]):
    people.extend(api.search_people(current_company=[company]))

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 1.34 s, sys: 83.5 ms, total: 1.43 s
Wall time: 19min 38s


In [ ]:
pprint(api.get_profile('daniela-dunn-950a45a3'))

{'certifications': [],
 'education': [{'degreeName': 'BCom (Hons)',
                'description': 'Completed courses in computer networks, '
                               'enterprise resource planning systems, '
                               'security, and information systems research.  '
                               'Wrote a dissertation on multicast network cost '
                               'allocation.\n'
                               '\n'
                               'Achieved first class distinction.',
                'entityUrn': 'urn:li:fs_education:(ACoAABYLkwwBZlmZPuUAK9ZUaloP8tnTfb1tUeo,239453468)',
                'fieldOfStudy': 'Information Systems',
                'school': {'active': True,
                           'entityUrn': 'urn:li:fs_miniSchool:15518',
                           'logoUrl': 'https://media-exp1.licdn.com/dms/image/C560BAQFcif2vOtbppQ/company-logo_',
                           'objectUrn': 'urn:li:school:15518',
                          

In [ ]:
df = pd.DataFrame(people).drop_duplicates()
df

,urn_id,distance,public_id,tracking_id
0,ACoAAAR370cBo_Bgfx0jX4qvqnKeqTDz4BeY88Y,DISTANCE_2,yen-shih-94713421,74968903
1,ACoAAAL-g0sBdldYgucu1UpPGQa3tuwyEHyOt3k,DISTANCE_2,pieta-brown-8639a614,50234187
2,ACoAACzdI1sBmzepHfdKrNSWAG6eT_DTt1nv6ZM,DISTANCE_2,matt-qiu-5b70a8190,752690011
3,ACoAAAv0TbYBFImKTXlP24EVJnpHZB5cwQHQcC8,DISTANCE_2,nicoleyuelin,200560054
4,ACoAAAHcI9sBTctUWYn90Kxl8V0MZrSqknow5TQ,DISTANCE_2,colinrossburnett,31204315
...,...,...,...,...
9468,ACoAAA0NmdMB3uuf2F_LXRAMeaWNS6mbmKec8FE,DISTANCE_2,michaellovegrove,218995155
9469,ACoAAB2Cz5kB4G_WpIILwIbWm1KXqzSaeoCmjGU,DISTANCE_2,steven-hung-kai-cheung-24998b119,495112089
9470,ACoAAA62h_MBNMMJFK37hevIawy3L5pYr_KAQ0w,DISTANCE_2,niharvasa,246843379
9473,ACoAAA0CvXYBLwGXYW5K1bcCY-gOXkco5cJ9SJk,DISTANCE_2,carriekllo,218283382


In [ ]:
print(f"To extract all {len(df)} profiles, at a rate of about 1 per second would take around {round(len(df) / 60 / 60, 2)} hours")

To extract all 6992 profiles, at a rate of about 1 per second would take around 1.94 hours


In [ ]:
try:
    profiles = json.load(open("profiles.json"))
except FileNotFoundError:
    profiles = []
print(len(profiles))

7440


In [ ]:
existing_ids = [p["profile_id"] for p in profiles]

In [65]:
new_profiles = df[~df.urn_id.isin(existing_ids)]
new_profiles

,urn_id,distance,public_id,tracking_id
65,ACoAAAuJUoABgm3BKFBC2n_Iy6sjeXwmEWqQcY0,DISTANCE_2,mitchell-parr-59499b54,193548928
136,ACoAACQUZsIBvOCJYsV8Y1qoLaJQ7WyBvsUofx4,DISTANCE_3,sheetalshankar94,605316802
280,ACoAAAialBcBH0Lt_nozSAzO_S527gpxgi27hig,DISTANCE_3,nikki-silvester-9b341240,144348183
365,ACoAAAKSSYIBGfzj7mbq_D0T5irSYDUfBHdl1is,DISTANCE_3,marthajaned,43141506
380,ACoAAAa4LiIBgoFo44EXAYG3EijYhSjLX-g3IC0,DISTANCE_3,jemmapenelopemisjp,112733730
...,...,...,...,...
9276,ACoAAA3WFrUBcoIyy7_rCUDS11yEvGHjDCp1Mgo,DISTANCE_2,williamkimca29,232134325
9281,ACoAAAQ0IXkBLShPKD4juJP3T3IHZBhCzpwr738,DISTANCE_2,weiou,70525305
9470,ACoAAA62h_MBNMMJFK37hevIawy3L5pYr_KAQ0w,DISTANCE_2,niharvasa,246843379
9473,ACoAAA0CvXYBLwGXYW5K1bcCY-gOXkco5cJ9SJk,DISTANCE_2,carriekllo,218283382


In [66]:
json.dump(people, open("people.json", "w"))

In [69]:
# LinkedIn appear to have hardened their anti-bot protection, making this kind of operation unfeasible
e = None
for public_id in tqdm(new_profiles.public_id):
    try:
        profiles.append(api.get_profile(public_id))
        #time.sleep(10)
    except Exception as e:
        print(e)
        raise

  0%|          | 0/2024 [00:00<?, ?it/s]

Expecting value: line 1 column 1 (char 0)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [70]:
len(profiles)

7791

In [71]:
json.dump(profiles, open("profiles.json", "w"))

In [91]:
# De-duplicate profiles
profile_dict = {}
for p in profiles:
    if "profile_id" in p:
        profile_dict[p["profile_id"]] = p
print(len(profile_dict))

7616


In [101]:
nodes = {}
edges = {}
for r in tqdm(profile_dict.values()):
    for i, e in enumerate(r["experience"]):
        companyName = e.get("companyName")
        if not companyName:
            pprint(e)
            continue
        if companyName not in nodes:
            industry = None
            industries = e.get("company", {}).get("industries")
            if industries:
                industry = industries[0]
            nodes[companyName] = {
                "id": e.get("companyUrn"),
                "name": companyName,
                "geoLocationName": e.get("geoLocationName"),
                "locationName": e.get("locationName"),
                "industry": industry,
                "companyLogoUrl": e.get("companyLogoUrl"),
                "val": 0
            }
        nodes[companyName]["val"] += 1
        if i < (len(r["experience"]) - 1):
            prevCompany = r["experience"][i + 1]
            prevCompanyName = prevCompany.get("companyName")
            if not prevCompanyName:
                pprint(prevCompany)
                continue
            edge_id = prevCompanyName + "_" + companyName
            if edge_id not in edges:
                edges[edge_id] = {
                    "source": prevCompanyName,
                    "target": companyName,
                    "val": 0
                }
            edges[edge_id]["val"] += 1

  0%|          | 0/7616 [00:00<?, ?it/s]

{'$anti_abuse_metadata': {'/companyName': {'sourceUrns': {'com.linkedin.common.urn.MemberUrn': 'urn:li:member:347225297'}},
                          '/companyUrn': {'sourceUrns': {'com.linkedin.common.urn.MemberUrn': 'urn:li:member:347225297'}},
                          '/description': {'sourceUrns': {'com.linkedin.common.urn.MemberUrn': 'urn:li:member:347225297'}},
                          '/entityUrn': {'sourceUrns': {'com.linkedin.common.urn.MemberUrn': 'urn:li:member:347225297'}},
                          '/geoLocationName': {'sourceUrns': {'com.linkedin.common.urn.MemberUrn': 'urn:li:member:347225297'}},
                          '/geoUrn': {'sourceUrns': {'com.linkedin.common.urn.MemberUrn': 'urn:li:member:347225297'}},
                          '/locationName': {'sourceUrns': {'com.linkedin.common.urn.MemberUrn': 'urn:li:member:347225297'}},
                          '/promotion': {'sourceUrns': {'com.linkedin.common.urn.MemberUrn': 'urn:li:member:347225297'}},
             

In [119]:
node_df = pd.DataFrame(nodes.values()).sort_values(by="val", ascending=False)
pd.set_option('display.max_rows', 100)
node_df.head(20)

,id,name,geoLocationName,locationName,industry,companyLogoUrl,val
1,urn:li:fs_miniCompany:3255299,PwC New Zealand,"Auckland, New Zealand","Auckland, New Zealand",Management Consulting,https://media-exp1.licdn.com/dms/image/C4D0BAQ...,2986
104,urn:li:fs_miniCompany:397575,KPMG New Zealand,"Auckland, New Zealand","Auckland, New Zealand",Management Consulting,https://media-exp1.licdn.com/dms/image/C560BAQ...,2655
176,urn:li:fs_miniCompany:1073,EY,"Auckland, New Zealand","Auckland, New Zealand",Accounting,https://media-exp1.licdn.com/dms/image/C510BAQ...,2457
151,urn:li:fs_miniCompany:1038,Deloitte,"Wellington, Wellington Region, New Zealand","Wellington, Wellington Region, New Zealand",Management Consulting,https://media-exp1.licdn.com/dms/image/C4E0BAQ...,1239
38,urn:li:fs_miniCompany:1038,Deloitte New Zealand,"Auckland, New Zealand","Auckland, New Zealand",Management Consulting,https://media-exp1.licdn.com/dms/image/C4E0BAQ...,1237
52,None,PwC,"London, England, United Kingdom","London, England, United Kingdom",None,None,470
3,urn:li:fs_miniCompany:3691,The University of Auckland,"Auckland, New Zealand","Auckland, New Zealand",Higher Education,https://media-exp1.licdn.com/dms/image/C560BAQ...,425
54,urn:li:fs_miniCompany:270126,ASB Bank,"Auckland, New Zealand","Auckland, New Zealand",Banking,https://media-exp1.licdn.com/dms/image/C560BAQ...,258
164,urn:li:fs_miniCompany:2437,ANZ,"Auckland, New Zealand","Auckland, New Zealand",Banking,https://media-exp1.licdn.com/dms/image/C560BAQ...,242
732,urn:li:fs_miniCompany:1073,Ernst & Young,None,None,Accounting,https://media-exp1.licdn.com/dms/image/C510BAQ...,225


In [120]:
node_df.industry.value_counts().head(50)

Information Technology and Services    732
Financial Services                     455
Accounting                             369
Management Consulting                  299
Retail                                 213
Computer Software                      210
Nonprofit Organization Management      209
Marketing and Advertising              177
Higher Education                       160
Education Management                   143
Government Administration              134
Hospital & Health Care                 130
Banking                                125
Law Practice                           123
Real Estate                            114
Construction                           110
Food & Beverages                       106
Insurance                               97
Hospitality                             96
Staffing and Recruiting                 95
Internet                                95
Telecommunications                      88
Health, Wellness and Fitness            87
Investment 

In [121]:
node_df[~pd.isna(node_df.geoLocationName) & ~pd.isna(node_df.locationName) & (node_df.geoLocationName!=node_df.locationName)]

,id,name,geoLocationName,locationName,industry,companyLogoUrl,val
4388,urn:li:fs_miniCompany:1038,Deloitte France,"Région de Paris, France","Paris Area, France",Management Consulting,https://media-exp1.licdn.com/dms/image/C4E0BAQ...,2
4611,urn:li:fs_miniCompany:163134,Coca-Cola FEMSA,"Ciudad de México y alrededores, México","Mexico City Area, Mexico",Food & Beverages,https://media-exp1.licdn.com/dms/image/C560BAQ...,2
7249,urn:li:fs_miniCompany:1508541,P.J. Clarke's,"São Paulo e Região, Brasil","São Paulo Area, Brazil",Restaurants,None,1
7248,None,Conselho Regional de Psicologia (Psychology Re...,"São Paulo e Região, Brasil","São Paulo Area, Brazil",None,None,1
6414,urn:li:fs_miniCompany:396744,The Hunting Dynasty,"Londres, Reino Unido","London, United Kingdom",Market Research,https://media-exp1.licdn.com/dms/image/C510BAQ...,1
8750,urn:li:fs_miniCompany:1734397,Grant Thornton International Ltd,"Bogotá D.C., Colombia","Bogotá D.C. Area, Colombia",Accounting,https://media-exp1.licdn.com/dms/image/C4E0BAQ...,1
9590,None,Mattias Jansson dressyrstall,"Lund, Sverige","Lund, Sweden",None,None,1
7687,urn:li:fs_miniCompany:1040979,JD.COM,中国 北京,"Beijing City, China",Internet,https://media-exp1.licdn.com/dms/image/C4E0BAQ...,1
8534,None,French Australian Chamber of Commerce & Indust...,"Région de Perth, Australie","Perth, Australia",None,None,1
8533,urn:li:fs_miniCompany:11305,LCL - BEGF,"Région de Aix-En-Provence, France","Aix-En-Provence Area, France",Banking,https://media-exp1.licdn.com/dms/image/C560BAQ...,1


In [122]:
print(f"{sum(node_df.geoLocationName.str.contains('New Zealand', na=False))} companies in NZ out of {len(node_df)}")

4384 companies in NZ out of 9865


In [123]:
edge_df = pd.DataFrame(edges.values()).sort_values(by="val", ascending=False)
edge_df.head(20)

,source,target,val
18,PwC New Zealand,PwC New Zealand,998
437,EY,EY,848
130,KPMG New Zealand,KPMG New Zealand,814
42,Deloitte New Zealand,Deloitte New Zealand,373
418,Deloitte,Deloitte,231
59,PwC,PwC,108
221,PwC,PwC New Zealand,105
268,Deloitte New Zealand,Deloitte,98
356,ANZ,ANZ,72
642,Fonterra,Fonterra,72


In [124]:
len(nodes), len(edges)

(9865, 18128)

In [125]:
network = {
    "nodes": list(nodes.values()),
    "links": list(edges.values())
}
json.dump(network, open("network.json", "w"))

In [126]:
%%time
with pd.ExcelWriter('linkedin.xlsx') as writer:
    node_df.to_excel(writer, sheet_name='Nodes', index=False)
    edge_df.to_excel(writer, sheet_name='Edges', index=False)

CPU times: user 2.04 s, sys: 106 ms, total: 2.14 s
Wall time: 2.14 s
